## Tech Challenge Fase 4 - Data Viz and Productions Models

In [33]:
# Bibliotecas

import pandas as pd
import numpy as np


In [34]:
# Importação do arquivo Obesity.csv

try:
    df_obesidade = pd.read_csv('Obesity.csv', delimiter=',')
except FileNotFoundError:
    print("Erro: Arquivo 'Obesity.csv' não encontrado. Por favor, verifique o caminho do arquivo.")
    exit()

In [37]:
# Padronização das colunas
df_obesidade.rename(columns={
    'Gender': 'genero',
    'Age': 'idade',
    'Height': 'altura',
    'Weight': 'peso',
    'family_history': 'hist_familiar',
    'FAVC': 'freq_alimentos_caloricos',
    'FCVC': 'freq_consumo_vegetais',
    'NCP': 'qtd_refeicoes_dia',
    'CAEC': 'alimentos_entre_refeicoes',
    'SMOKE': 'flag_fumo',
    'CH2O': 'qtd_dia_agua',
    'SCC': 'monitoramento_calorias',
    'FAF': 'freq_ativ_fisica',
    'TUE': 'freq_uso_tech',
    'CALC': 'freq_alcool',
    'MTRANS': 'meio_transporte',
    'Obesity': 'nivel_obesidade'
}, inplace=True)

In [38]:
df_obesidade.head()

,genero,idade,altura,peso,hist_familiar,freq_alimentos_caloricos,freq_consumo_vegetais,qtd_refeicoes_dia,alimentos_entre_refeicoes,flag_fumo,qtd_dia_agua,monitoramento_calorias,freq_ativ_fisica,freq_uso_tech,freq_alcool,meio_transporte,nivel_obesidade
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


# --- 2. Engenharia de Features (Criação de Novas Variáveis) ---

In [39]:
# Feature 1: Índice de Massa Corporal (IMC)
# Fórmula: peso (kg) / [altura (m)]^2
# É um indicador universalmente aceito para níveis de gordura corporal.

df_obesidade['imc'] = df_obesidade['peso'] / (df_obesidade['altura'] ** 2)

In [40]:
# Feature 2: Categorização do IMC
# Criar categorias de IMC baseadas nos padrões da OMS para dar um contexto clínico imediato.

bins = [0, 18.5, 24.9, 29.9, 34.9, 39.9, np.inf]
labels = ['Abaixo do Peso', 'Peso Normal', 'Sobrepeso', 'Obesidade Grau I', 'Obesidade Grau II', 'Obesidade Grau III']
df_obesidade['categoria_imc'] = pd.cut(df_obesidade['imc'], bins=bins, labels=labels, right=False)

In [41]:
# Feature 3: Score de Hábitos de Vida (Lifestyle Score)
# Um indicador composto para avaliar o quão saudável é o estilo de vida do paciente.
# Vamos atribuir pontos para hábitos positivos e negativos.
# Pontos:
# - Consumo de vegetais: 1, 2, 3 -> +1, +2, +3
# - Atividade física: 0, 1, 2, 3 -> +0, +1, +2, +3
# - Consumo de água: 1, 2, 3 -> +1, +2, +3
# - Comida calórica: 'yes' -> -2, 'no' -> +2
# - Fumar: 'yes' -> -2, 'no' -> +1
# - Monitoramento de calorias: 'yes' -> +2, 'no' -> -1

lifestyle_score = (
    df_obesidade['freq_consumo_vegetais'].round() +
    df_obesidade['freq_ativ_fisica'].round() +
    df_obesidade['qtd_dia_agua'].round() +
    df_obesidade['freq_alimentos_caloricos'].apply(lambda x: 2 if x == 'no' else -2) +
    df_obesidade['flag_fumo'].apply(lambda x: 1 if x == 'no' else -2) +
    df_obesidade['monitoramento_calorias'].apply(lambda x: 2 if x == 'yes' else -1)
)
df_obesidade['lifestyle_score'] = lifestyle_score

In [42]:
# Feature 4: Relação Cintura-Altura (RCA) - Estimativa
# Conceito: Embora não tenhamos a medida da cintura, o uso de tecnologia (TUE) e o modo de transporte (MTRANS) 
# podem ser proxies para um estilo de vida sedentário, que está associado à gordura abdominal.
# Criação de "Índice de Sedentarismo".
# Pontos:
# - Tempo de uso de tecnologia (time_using_tech): 0, 1, 2 -> +0, +1, +2
# - Modo de transporte (transportation_mode): 'Walking'/'Bike' -> -1, 'Public' -> +1, 'Automobile' -> +2
transport_map = {'Walking': -1, 'Bike': -1, 'Public_Transportation': 1, 'Motorbike': 1, 'Automobile': 2}
df_obesidade['indice_sedentarismo'] = df_obesidade['freq_uso_tech'].round() + df_obesidade['meio_transporte'].map(transport_map)


# --- 3. Tratamento e Formatação dos Dados ---

In [43]:
# Traduzir os valores de algumas colunas para português para o Power BI ficar mais claro.
df_powerbi = df_obesidade.copy()

In [44]:
# Mapeamentos para tradução
yes_no_map = {'yes': 'Sim', 'no': 'Não'}
gender_map = {'Male': 'Masculino', 'Female': 'Feminino'}
food_between_meals_map = {'no': 'Não', 'Sometimes': 'Às vezes', 'Frequently': 'Frequentemente', 'Always': 'Sempre'}
alcohol_map = {'no': 'Não', 'Sometimes': 'Às vezes', 'Frequently': 'Frequentemente'}
transport_map_pt = {
    'Automobile': 'Automóvel',
    'Motorbike': 'Moto',
    'Bike': 'Bicicleta',
    'Public_Transportation': 'Transporte Público',
    'Walking': 'Caminhada'
}
obesity_level_map_pt = {
    'Normal_Weight': 'Peso Normal',
    'Overweight_Level_I': 'Sobrepeso Nível I',
    'Overweight_Level_II': 'Sobrepeso Nível II',
    'Obesity_Type_I': 'Obesidade Tipo I',
    'Obesity_Type_II': 'Obesidade Tipo II',
    'Obesity_Type_III': 'Obesidade Tipo III',
    'Insufficient_Weight': 'Peso Insuficiente'
}

In [45]:
# Aplicando as traduções
df_powerbi['hist_familiar'] = df_powerbi['hist_familiar'].map(yes_no_map)
df_powerbi['freq_alimentos_caloricos'] = df_powerbi['freq_alimentos_caloricos'].map(yes_no_map)
df_powerbi['flag_fumo'] = df_powerbi['flag_fumo'].map(yes_no_map)
df_powerbi['monitoramento_calorias'] = df_powerbi['monitoramento_calorias'].map(yes_no_map)
df_powerbi['genero'] = df_powerbi['genero'].map(gender_map)
df_powerbi['alimentos_entre_refeicoes'] = df_powerbi['alimentos_entre_refeicoes'].map(food_between_meals_map)
df_powerbi['freq_alcool'] = df_powerbi['freq_alcool'].map(food_between_meals_map)
df_powerbi['meio_transporte'] = df_powerbi['meio_transporte'].map(transport_map_pt)
df_powerbi['nivel_obesidade'] = df_powerbi['nivel_obesidade'].map(obesity_level_map_pt)

In [48]:
# Arredondar valores numéricos para melhor visualização
cols_to_round = ['idade', 'altura', 'peso', 'imc']
df_powerbi[cols_to_round] = df_powerbi[cols_to_round].round(2)

# --- 4. Salvar o Arquivo Processado ---

In [49]:
# O arquivo `analise_obesidade.csv` será usado no Power BI.
# O arquivo `dados_modelo.csv` (o original, sem traduções) será usado para o modelo.
output_filename_powerbi = 'analise_obesidade.csv'
output_filename_model = 'dados_modelo.csv'

df_powerbi.to_csv(output_filename_powerbi, index=False, encoding='utf-8-sig')
df_obesidade.to_csv(output_filename_model, index=False, encoding='utf-8')

print(f"Engenharia de features concluída com sucesso!")
print(f"Arquivo para análise no Power BI salvo como: '{output_filename_powerbi}'")
print(f"Arquivo para treinamento do modelo salvo como: '{output_filename_model}'")
print("\nVisualização das primeiras linhas do dataframe processado:")
print(df_powerbi.head())

Engenharia de features concluída com sucesso!
Arquivo para análise no Power BI salvo como: 'analise_obesidade.csv'
Arquivo para treinamento do modelo salvo como: 'dados_modelo.csv'

Visualização das primeiras linhas do dataframe processado:
      genero  idade  altura  peso hist_familiar freq_alimentos_caloricos  \
0   Feminino   21.0    1.62  64.0           Sim                      Não   
1   Feminino   21.0    1.52  56.0           Sim                      Não   
2  Masculino   23.0    1.80  77.0           Sim                      Não   
3  Masculino   27.0    1.80  87.0           Não                      Não   
4  Masculino   22.0    1.78  89.8           Não                      Não   

   freq_consumo_vegetais  qtd_refeicoes_dia alimentos_entre_refeicoes  \
0                    2.0                3.0                  Às vezes   
1                    3.0                3.0                  Às vezes   
2                    2.0                3.0                  Às vezes   
3         